In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install lifelines -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 7.7 MB/s eta 0:00:00


In [ ]:
TRAIN_PATH = "/content/drive/MyDrive/1:1_Shriya_Wagholikar/Datasets/experiment_1/exp1_selectKBest_vif_train.csv"
TEST_PATH = "/content/drive/MyDrive/1:1_Shriya_Wagholikar/Datasets/experiment_1/exp1_selectKBest_vif_test.csv"

In [ ]:
import pandas as pd
train_data = pd.read_csv(TRAIN_PATH)
train_data.head()

,ASB13,C6orf58,TMEM248,DNAJB12,EN1,FAM114A1,FAM46A,GALNT2,GGCX,GNG12,...,SKI,TCEAL1,TCEAL2,TEF,TMEM194B,TNFRSF11B,TUBGCP2,WISP1,Disease Free (Months),DFS_STATUS_ENCODED
0,-1.8265,-0.2004,2.3701,-0.6147,1.3637,0.6627,-0.5268,0.0339,1.4849,1.0234,...,0.1266,0.3299,0.4246,-0.0560,0.5439,1.6735,-0.8906,-0.9555,2.43,0.0
1,-0.4475,-0.2125,-1.4078,-0.0811,-0.0143,-1.6606,-1.5210,-2.3393,-1.2223,-0.7103,...,0.6464,2.2317,0.7983,-0.2006,-0.1001,-0.0141,0.7203,-1.2828,20.20,1.0
2,-0.3157,-0.3844,-0.5240,0.1379,-0.0853,0.1569,-1.1803,-0.3884,-1.2780,-0.3746,...,-0.3653,1.0418,1.2324,1.6264,-0.3695,-0.0955,0.2028,0.4800,20.43,0.0
3,0.3588,-0.9091,-1.0282,0.5796,-0.4008,-0.2822,0.1314,-0.5088,-0.0667,-0.7933,...,-0.9298,-0.2155,0.4715,0.7224,-0.4064,-1.2270,0.5372,-0.7155,8.18,0.0
4,1.2976,-1.9452,-0.5185,0.5071,-1.0241,-1.2173,1.1226,-0.1790,-0.7596,-0.5763,...,-0.7926,1.2075,0.6741,1.1507,-1.5549,-0.6189,1.0314,-0.4206,31.80,0.0


In [ ]:
test_data = pd.read_csv(TEST_PATH)
test_data.head()

,ASB13,C6orf58,TMEM248,DNAJB12,EN1,FAM114A1,FAM46A,GALNT2,GGCX,GNG12,...,SKI,TCEAL1,TCEAL2,TEF,TMEM194B,TNFRSF11B,TUBGCP2,WISP1,Disease Free (Months),DFS_STATUS_ENCODED
0,-0.1050,2.1691,-0.6761,0.1240,-0.9673,-0.9148,-1.0645,-0.3780,-1.5448,-0.2980,...,-0.4457,0.7045,0.4209,0.6173,-1.0707,-0.5709,1.3146,-0.2024,119.48,1.0
1,-0.5950,-0.2513,-0.2209,1.2554,-0.9815,0.2250,0.7169,0.9993,0.4485,0.2041,...,1.3895,0.1068,0.8163,-0.1429,0.6752,0.0074,0.8986,-0.3668,95.86,0.0
2,0.2187,-1.9452,-0.3263,0.3733,0.2466,0.3526,1.4144,0.3906,1.2149,0.4633,...,0.6493,-0.0159,-0.2995,-1.4228,-0.4799,1.5829,0.3286,-0.5494,16.79,1.0
3,0.5543,1.1511,-0.6104,-0.1870,-0.7498,-0.0701,-1.5171,-1.0341,-1.0295,0.0027,...,-0.0380,0.4263,0.0930,0.9072,-0.8850,-0.2086,0.3464,-0.2734,14.55,0.0
4,0.3029,0.6479,-0.7701,0.5613,-0.5275,-0.1509,0.2339,-0.4920,0.0690,-1.3476,...,-0.7415,-0.4040,0.1320,0.7651,-0.3531,-0.7662,0.7937,0.8867,31.18,0.0


In [ ]:
# #in the event you have to drop columns -> only execute if you have some columns to drop in step 2, which we don't have currently
# cols_to_drop = []
# train_data.drop(cols_to_drop, axis=1, inplace=True)
# test_data.drop(cols_to_drop, axis=1, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split, KFold
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
import numpy as np
def apply_cross_validation(df:pd.DataFrame, penalty: float = 0.1) -> None:
    # create the folds
    kf = KFold(n_splits=5, shuffle=True, random_state=42) #this is a 5 cross validation
    c_index_scores = []
    # run cross validation
    for train_index, val_index in kf.split(train_data):
        train_fold = df.iloc[train_index]
        val_fold = df.iloc[val_index]
        cph = CoxPHFitter(penalizer = penalty)
        cph.fit(train_fold, duration_col='Disease Free (Months)', event_col='DFS_STATUS_ENCODED')
        val_predictions = cph.predict_partial_hazard(val_fold)
        c_index = concordance_index(val_fold['Disease Free (Months)'], -val_predictions, val_fold['DFS_STATUS_ENCODED'])
        c_index_scores.append(c_index)
    print(f"Average Validation C-index: {np.mean(c_index_scores)}")

In [ ]:
apply_cross_validation(train_data, penalty = 0.7)

Average Validation C-index: 0.7430148323464276


In [ ]:
# Train the final model on the entire training set with penelizer giving best C-index
cph_final = CoxPHFitter(penalizer = 1) #change penalizer value starting from 0 to 1 by increments of .1
# fit on train data
cph_final.fit(train_data, duration_col='Disease Free (Months)', event_col='DFS_STATUS_ENCODED')
cph_final.print_summary()

<lifelines.CoxPHFitter: fitted with 382 total observations, 250 right-censored observations>
             duration col = 'Disease Free (Months)'
                event col = 'DFS_STATUS_ENCODED'
                penalizer = 1
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 382
number of events observed = 132
   partial log-likelihood = -606.63
         time fit was run = 2025-09-26 00:34:32 UTC

---
           coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                  
ASB13     -0.05      0.95      0.05           -0.14            0.05                0.87                1.05
C6orf58   -0.05      0.95      0.04           -0.13            0.03                0.88                1.03
TMEM248    0.01      1.01      0.05           -0.08            0.10                0.92                1.11
DNAJB12   -0.04      0.96      0.05           -0.13            0.05                0.88                1.06
EN1        0.06      1.07      0.05           -0.03            0.16                0.97                1.17
FAM114A1   0.01      1.01      0.05           -0.09            0.11                0.91                1.11
FAM46A     0.05      1.05      0.05           -0.05            0.14                0.95                1.15
GALNT2     0.01      1.01      0.05           -0.09            0.10                0.92                1.11
GGCX       0.04      1.04      0.05           -0.06            0.13                0.94                1.14
GNG12      0.02      1.02      0.05           -0.07            0.11                0.93                1.12
DNAAF5     0.03      1.03      0.05           -0.06            0.12                0.94                1.13
IDI1      -0.01      0.99      0.05           -0.10            0.08                0.90                1.09
CAMSAP3   -0.03      0.97      0.05           -0.13            0.06                0.88                1.06
LOC400043  0.04      1.04      0.05           -0.05            0.13                0.95                1.14
LRIT2     -0.01      0.99      0.05           -0.11            0.08                0.90                1.09
MAGEH1    -0.03      0.97      0.05           -0.13            0.06                0.88                1.06
MRFAP1L1  -0.06      0.94      0.05           -0.15            0.04                0.86                1.04
NGFRAP1   -0.01      0.99      0.05           -0.10            0.08                0.90                1.09
PSTK      -0.05      0.96      0.05           -0.14            0.05                0.87                1.05
RPA1       0.02      1.02      0.05           -0.07            0.11                0.93                1.11
SH2D4A     0.05      1.05      0.05           -0.05            0.14                0.95                1.16
SIPA1L3    0.03      1.03      0.05           -0.06            0.12                0.94                1.13
SKI        0.02      1.02      0.05           -0.07            0.11                0.93                1.12
TCEAL1    -0.02      0.98      0.05           -0.12            0.08                0.88                1.08
TCEAL2    -0.04      0.96      0.05           -0.14            0.06                0.87                1.06
TEF       -0.03      0.97      0.05           -0.12            0.07                0.88                1.07
TMEM194B   0.05      1.05      0.05           -0.04            0.14                0.96                1.15
TNFRSF11B  0.04      1.04      0.05           -0.06            0.13                0.94                1.14
TUBGCP2   -0.02      0.98      0.05           -0.12            0.08                0.89                1.08
WISP1      0.05      1.05      0.05           -0.04            0.14                0.96                1.15

           cmp to     z    p  -log2(p)
covariate                             
ASB13        0.0

In [ ]:
# Concordance Index
c_index = cph_final.concordance_index_
print(f"Concordance Index: {c_index}")

Concordance Index: 0.7600642081696447


Saving the Best Model

In [ ]:
# Train the final model on the entire training set with penelizer giving best C-index
cph_final = CoxPHFitter(penalizer = 0)
# fit on train data
cph_final.fit(train_data, duration_col='Disease Free (Months)', event_col='DFS_STATUS_ENCODED')
cph_final.print_summary()

<lifelines.CoxPHFitter: fitted with 382 total observations, 250 right-censored observations>
             duration col = 'Disease Free (Months)'
                event col = 'DFS_STATUS_ENCODED'
      baseline estimation = breslow
   number of observations = 382
number of events observed = 132
   partial log-likelihood = -586.85
         time fit was run = 2025-10-09 23:38:34 UTC

---
           coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                  
ASB13     -0.15      0.86      0.16           -0.47            0.17                0.62                1.19
C6orf58   -0.29      0.75      0.11           -0.50           -0.08                0.61                0.92
TMEM248   -0.21      0.81      0.18           -0.57            0.14                0.56                1.15
DNAJB12   -0.14      0.87      0.17           -0.46            0.19                0.63                1.21
EN1        0.17      1.19      0.14           -0.10            0.45                0.91                1.56
FAM114A1  -0.21      0.81      0.18           -0.57            0.15                0.57                1.16
FAM46A     0.16      1.17      0.14           -0.11            0.43                0.89                1.54
GALNT2    -0.13      0.88      0.17           -0.47            0.21                0.62                1.23
GGCX       0.13      1.14      0.17           -0.20            0.46                0.82                1.58
GNG12      0.03      1.03      0.19           -0.34            0.40                0.71                1.49
DNAAF5     0.11      1.11      0.17           -0.22            0.43                0.81                1.54
IDI1       0.16      1.17      0.14           -0.11            0.42                0.89                1.53
CAMSAP3    0.02      1.02      0.15           -0.27            0.31                0.76                1.36
LOC400043  0.21      1.23      0.14           -0.07            0.48                0.93                1.62
LRIT2      0.25      1.28      0.20           -0.14            0.63                0.87                1.89
MAGEH1    -0.08      0.92      0.13           -0.34            0.18                0.71                1.19
MRFAP1L1  -0.20      0.82      0.13           -0.44            0.05                0.64                1.05
NGFRAP1    0.12      1.13      0.14           -0.16            0.41                0.85                1.50
PSTK      -0.15      0.86      0.14           -0.42            0.12                0.66                1.13
RPA1       0.09      1.09      0.13           -0.16            0.34                0.85                1.41
SH2D4A     0.23      1.26      0.17           -0.11            0.57                0.90                1.77
SIPA1L3    0.12      1.13      0.14           -0.16            0.40                0.85                1.49
SKI        0.04      1.04      0.14           -0.24            0.31                0.79                1.36
TCEAL1    -0.05      0.95      0.16           -0.36            0.26                0.70                1.30
TCEAL2    -0.14      0.87      0.19           -0.51            0.22                0.60                1.25
TEF        0.00      1.00      0.15           -0.30            0.30                0.74                1.35
TMEM194B   0.13      1.14      0.14           -0.14            0.41                0.87                1.50
TNFRSF11B  0.02      1.02      0.17           -0.31            0.35                0.73                1.41
TUBGCP2    0.07      1.07      0.17           -0.27            0.40                0.77                1.49
WISP1      0.01      1.01      0.14           -0.26            0.28                0.77                1.32

           cmp to     z    p  -log2(p)
covariate                             
ASB13        0.00 -0.92 0.36      1.49
C6orf58      0.00 -2.76 0.01      7.42


In [ ]:
# Concordance Index
c_index = cph_final.concordance_index_
print(f"Concordance Index: {c_index}")

Concordance Index: 0.7752291640265281


In [ ]:
import pickle
MODEL_PATH = "/content/drive/MyDrive/1:1_Shriya_Wagholikar/Results/experiment_1/best_model_exp1"
with open(MODEL_PATH, 'wb') as file:
    pickle.dump(cph_final, file)

In [ ]:
# define the train and test path
UPDATED_TRAIN_PATH = "/content/drive/MyDrive/1:1_Shriya_Wagholikar/Datasets/experiment_1/updated_exp1_selectKBest_vif_train.csv"
UPDATED_TEST_PATH = "/content/drive/MyDrive/1:1_Shriya_Wagholikar/Datasets/experiment_1/updated_exp1_selectKBest_vif_test.csv"
train_data.to_csv(UPDATED_TRAIN_PATH, index=False)
test_data.to_csv(UPDATED_TEST_PATH, index=False)